# Lab 5 Report

Prepared by Zech Wolf

## 1. Data preparation

I use the personal key indicators of heart disease dataset found on Kaggle. The data has a mix of continuous and categorical features, and contains over 300,000 instances. The target variable is "HeartDisease" which indicates whether or not a patient has experienced heart disease. This will be label encoded as 0 or 1.

There are 17 features in the dataset, both categorical and continuous. The categorical features will be label encoded as integers to support embedding. The continuous features will be processed through a standard scaler.

Dataset source: https://www.kaggle.com/datasets/kamilpytlak/personal-key-indicators-of-heart-disease?resource=download

### 1.1 Preprocessing

In [6]:
import pandas as pd

df = pd.read_csv("datasets/heart_2020_cleaned.csv")

#Quality check
df.isna().any()

HeartDisease        False
BMI                 False
Smoking             False
AlcoholDrinking     False
Stroke              False
PhysicalHealth      False
MentalHealth        False
DiffWalking         False
Sex                 False
AgeCategory         False
Race                False
Diabetic            False
PhysicalActivity    False
GenHealth           False
SleepTime           False
Asthma              False
KidneyDisease       False
SkinCancer          False
dtype: bool

In [8]:
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


#### Feature summary

| __Feature__ | __Type__ | __Representation__ |
|----|----|----|
| BMI | continuous | float |
| Smoking | categorical | int |
| AlcoholDrinking | categorical | int |
| Stroke | categorical | int |
| PhysicalHealth | continuous | float |
| MentalHealth | continuous | float |
| DiffWalking | categorical | int |
| Sex | categorical | int |
| AgeCategory | ordinal | int |
| Race | categorical | int |
| Diabetic | categorical | int |
| PhysicalActivity | categorical | int |
| GenHealth | ordinal | int |
| SleepTime | continuous | float |
| Asthma | categorical | int |
| KidneyDisease | categorical | int |
| SkinCancer | categorical | int |
| HeartDisease (target) | categorical | int |

#### Label encoding

There are quite a few categorical and ordinal features given in string format. These need to be replaced with integers in order to create embeddings later on.

In [10]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder

cat_columns = ["Smoking", "AlcoholDrinking", "Stroke", "DiffWalking",
                "Sex", "Race", "Diabetic", "PhysicalActivity", "Asthma",
                "KidneyDisease", "SkinCancer"]
ord_columns = ["GenHealth", "AgeCategory"]

le = dict()
oe = dict()

for col in cat_columns:
    enc = LabelEncoder()
    df[col] = enc.fit_transform(df[col])
    le[col] = enc #store in dict with col name as key




NameError: name 'df' is not defined

In [11]:
df.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,0,16.60,1,0,0,3.0,30.0,0,Female,55-59,White,1,1,Very good,5.0,1,0,1
1,0,20.34,0,0,1,0.0,0.0,0,Female,80 or older,White,0,1,Very good,7.0,0,0,0
2,0,26.58,1,0,0,20.0,30.0,0,Male,65-69,White,1,1,Fair,8.0,1,0,0
3,0,24.21,0,0,0,0.0,0.0,0,Female,75-79,White,0,0,Good,6.0,0,0,1
4,0,23.71,0,0,0,28.0,0.0,1,Female,40-44,White,0,1,Very good,8.0,0,0,0


#### Feature normalization

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()